In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('../../raw_data/State_Population_Data_2000_2019.xls', sheet_name="Sheet1")

In [3]:
data.head()

,observation_date,state,population
0,1950-01-01,ak,135.0
1,1950-01-01,al,3058.0
2,1950-01-01,ar,1908.0
3,1950-01-01,az,756.0
4,1950-01-01,ca,10677.0


In [4]:
data['day'] = data['observation_date'].apply(lambda x: x.strftime("%Y-%m-%d"))
data['month'] = data['observation_date'].apply(lambda x: x.strftime("%Y-%m"))
data['month_number'] = data['observation_date'].apply(lambda x: int(x.month))
data['year_number'] = data['observation_date'].apply(lambda x: int(x.year))

In [5]:
seasons = ['winter','winter','spring','spring','spring','summer','summer','summer','autumn','autumn','autumn','winter']
month_to_season = dict(zip(range(1,13), seasons))
data['season'] = data.month_number.map(month_to_season)
data['year_season'] = data['year_number'].astype(str)+'-'+data['season']

In [6]:
data.head()

,observation_date,state,population,day,month,month_number,year_number,season,year_season
0,1950-01-01,ak,135.0,1950-01-01,1950-01,1,1950,winter,1950-winter
1,1950-01-01,al,3058.0,1950-01-01,1950-01,1,1950,winter,1950-winter
2,1950-01-01,ar,1908.0,1950-01-01,1950-01,1,1950,winter,1950-winter
3,1950-01-01,az,756.0,1950-01-01,1950-01,1,1950,winter,1950-winter
4,1950-01-01,ca,10677.0,1950-01-01,1950-01,1,1950,winter,1950-winter


In [7]:
data.population.unique()

array([ 135.   , 3058.   , 1908.   , ..., 5832.655, 1784.787,  582.328])

In [8]:
def value_count_func (series_):
    return len(series_.value_counts())

def state_season_df (data_, state_):
    
    data_state = data_[data_['state'] == state_]
    
    data_state_season = data_state.groupby(by='year_season').agg({'population':'first'})
       
    data_state_season['state'] = state_
    
    return data_state_season.reset_index().set_index('state')

In [9]:
STATES_ = list(data.state.unique()) 
x_ = pd.DataFrame()

for st_ in STATES_:
    
    y_ = state_season_df (data, st_)
    x_ = pd.concat([x_,y_])
    
x_ = x_.reset_index()

In [10]:
x_[['year', 'season']] = x_['year_season'].str.split('-',expand=True)
x_ = x_[x_['year']>'1974']

In [11]:
x_.population.unique()

array([376.17 , 400.969, 403.436, ..., 579.054, 580.116, 582.328])

In [12]:
years_ = list(data.year_number.unique())

for yr_ in years_:
    c_ = data.loc[data['year'] == 'yr_']
    population_ = np.where(c_season == 'winter')
    c_['population'] = np.where((c_.population == 0),population_, c_.population)

In [ ]:
x_.to_csv('../../raw_data/population_by_state_season.csv',header=True,index=False)